In [ ]:
# Import thư viện
import numpy as np
import tritonclient.http as httpclient
from torchvision import transforms
from PIL import Image

In [ ]:
# Đường dẫn ảnh
image_path = "10.jpg"

In [ ]:
# Tiền xử lý ảnh
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Đọc và xử lý ảnh
img = Image.open(image_path).convert("RGB")
processed_img = preprocess(img).unsqueeze(0).numpy()

# Tạo input cho Triton
input_data = httpclient.InferInput("data_0", processed_img.shape, "FP32")
input_data.set_data_from_numpy(processed_img, binary_data=True)

In [ ]:
# Yêu cầu output
output_data = httpclient.InferRequestedOutput("fc6_1", binary_data=True, class_count=1000)

# Kết nối và gửi truy vấn đến Triton
triton_client = httpclient.InferenceServerClient("localhost:8000")
response = triton_client.infer(model_name="densenet_onnx", inputs=[input_data], outputs=[output_data])

In [ ]:
# Lấy kết quả và in ra 5 giá trị đầu tiên
output_result = response.as_numpy("fc6_1").astype(str)
print(np.squeeze(output_result)[:5])